# 3k Human Peripheral Blood Mononuclear Cells (PBMCs)

* [reference](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html)

## Download dataset

In [ ]:
!mkdir data
!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
!cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz
!mkdir write

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor="white")

## Load dataset

In [ ]:
results_file = "write/pbmc3k.h5ad"

In [ ]:
adata = sc.read_10x_mtx(
    "data/filtered_gene_bc_matrices/hg19/",  # the directory with the `.mtx` file
    var_names="gene_symbols",                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata

## Preprocessing

### Basic filtering

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata

### Show top 20 highly expressed genes

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

### Quality control

In [ ]:
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

### Filter out bad quality data

Maybe death cell or mock genes.

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

In [ ]:
adata

### Total-count normalization

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

### Logarithm

In [ ]:
sc.pp.log1p(adata)

### Identify highly-variable genes

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata)

### Save current data to `.raw` for later use in differential testing and visualizations of gene expression

In [ ]:
adata.raw = adata

In [ ]:
adata = adata[:, adata.var.highly_variable]

### Regress out

In [ ]:
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])

### Scaling

In [ ]:
sc.pp.scale(adata, max_value=10)

## PCA for denoising

In [ ]:
sc.tl.pca(adata, svd_solver="arpack")

In [ ]:
sc.pl.pca(adata, color="CST3")

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

### Save results

In [ ]:
adata

In [ ]:
adata.write(results_file)

## Computing the neighborhood graph

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

## Data visualization

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"])

In [ ]:
sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"], use_raw=False)

## Clustering

In [ ]:
sc.tl.leiden(adata)  # you can also use .louvain for Louvain clustering

In [ ]:
sc.pl.umap(adata, color=["leiden", "CST3", "NKG7"])

## Differentially expressed genes

In [ ]:
sc.tl.rank_genes_groups(adata, "leiden", method="t-test")

In [ ]:
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
# try method="wilcoxon" for Wilcoxon rank-sum test

In [ ]:
# try method="logreg" for logistic regression

## Define marker genes

In [ ]:
marker_genes = ["IL7R", "CD79A", "MS4A1", "CD8A", "CD8B", "LYZ", "CD14",
                "LGALS3", "S100A8", "GNLY", "NKG7", "KLRB1",
                "FCGR3A", "MS4A7", "FCER1A", "CST3", "PPBP"]

### Show top-10 ranked genes for each cluster

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)

In [ ]:
result = adata.uns["rank_genes_groups"]
groups = result["names"].dtype.names
pd.DataFrame(
    {group + "_" + key[:1]: result[key][group]
    for group in groups for key in ["names", "pvals"]}).head(5)

### Compare to specific cluster

In [ ]:
sc.tl.rank_genes_groups(adata, "leiden", groups=["0"], reference="1", method="wilcoxon")
sc.pl.rank_genes_groups(adata, groups=["0"], n_genes=20)

### Detailed gene expression level information

In [ ]:
sc.pl.rank_genes_groups_violin(adata, groups="0", n_genes=8)

In [ ]:
sc.pl.violin(adata, ["CST3", "NKG7", "PPBP"], groupby="leiden")

### Mark clusters

In [ ]:
new_cluster_names = [
    "CD4 T", "CD14 Monocytes",
    "B", "CD8 T",
    "NK", "FCGR3A Monocytes",
    "Dendritic", "Megakaryocytes"]
adata.rename_categories("leiden", new_cluster_names)

In [ ]:
sc.pl.umap(adata, color="leiden", legend_loc="on data", title="", frameon=False)

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby="leiden")

In [ ]:
adata

In [ ]:
adata.write(results_file, compression='gzip')